<a href="https://colab.research.google.com/github/rociavl/UNI/blob/main/P3_OP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import networkx as nx

def crear_grafo(precedencias, tiempos_proyecto):
    # Crear el grafo dirigido
    G_proyecto = nx.DiGraph()

    # Agregar un nodo ficticio inicial
    nodo_inicial_ficticio = 'α'
    for tarea, predecesores in precedencias.items():
        if not predecesores:  # Si la tarea no tiene predecesores
            G_proyecto.add_edge(nodo_inicial_ficticio, tarea, weight=0)  # Conectar al nodo ficticio con peso 0
        for predecesor in predecesores:
            G_proyecto.add_edge(predecesor, tarea, weight=tiempos_proyecto[tarea])

    # Añadir nodo ficticio final
    nodo_final_ficticio = 'ω'
    for tarea in list(G_proyecto.nodes):
        if G_proyecto.out_degree(tarea) == 0 and tarea != nodo_inicial_ficticio:
            G_proyecto.add_edge(tarea, nodo_final_ficticio, weight=0)

    # Añadir tiempos de procesamiento como atributos de nodos
    nx.set_node_attributes(G_proyecto, tiempos_proyecto, 'duration')
    camino_critico = nx.dag_longest_path(G_proyecto)

    return G_proyecto, camino_critico

def recalcular_tiempos(G, tiempos_proyecto):
    """Función para recalcular los tiempos de inicio y final para el grafo modificado"""

    # Calcular Early Start (ES) y Early Finish (EF)
    early_start = {nodo: 0 for nodo in tiempos_proyecto}  # Solo nodos reales

    for nodo in nx.topological_sort(G):  # Orden topológico
        if nodo in tiempos_proyecto:  # Solo calcular para nodos reales
            for predecesor in G.predecessors(nodo):
                if predecesor in early_start:  # Verificar que el predecesor esté en early_start
                    early_start[nodo] = max(early_start[nodo], early_start[predecesor] + tiempos_proyecto[predecesor])

    early_finish = {nodo: early_start[nodo] + tiempos_proyecto[nodo] for nodo in early_start}

    # Duración total del proyecto es el máximo tiempo de finalización
    duracion_total_proyecto = max(early_finish.values())

    # Calcular Late Finish (LF) y Late Start (LS) hacia atrás
    late_finish = {nodo: duracion_total_proyecto for nodo in tiempos_proyecto}  # Solo nodos reales

    for nodo in reversed(list(nx.topological_sort(G))):  # Invertimos el orden topológico
        if nodo in tiempos_proyecto:  # Solo calcular para nodos reales
            for sucesor in G.successors(nodo):
                if sucesor in late_finish:  # Verificar que el sucesor esté en late_finish
                    late_finish[nodo] = min(late_finish[nodo], late_finish[sucesor] - tiempos_proyecto[sucesor])

    late_start = {nodo: late_finish[nodo] - tiempos_proyecto[nodo] for nodo in late_finish}


    return early_start, early_finish, late_start, late_finish, duracion_total_proyecto

def incompatibilidad(node1, node2, early_start, late_start, tiempos_proyecto, duracion_total_proyecto):
    """Calcula el impacto en la duración total del proyecto si se cambia la precedencia entre node1 y node2"""
    # D_max calcula el nuevo tiempo para ambas opciones
    D_max = {
        'Right': early_start[node1] + tiempos_proyecto[node1] + (duracion_total_proyecto - late_start[node2]),
        'Left': early_start[node2] + tiempos_proyecto[node2] + (duracion_total_proyecto - late_start[node1])
    }

    mejor_opcion = min(D_max, key=D_max.get)  # Mejor opción
    respuesta = f'{node1} -> {node2}' if mejor_opcion == 'Right' else f'{node2} -> {node1}'

    # Ahorro calculado
    ahorro = max(D_max.values()) - min(D_max.values())
    return min(D_max.values()), respuesta, ahorro

def comp_incompatibilidad(Casos, G, early_start, late_start, tiempos_proyecto, duracion_total_proyecto):
    """Función que evalúa todas las incompatibilidades y selecciona la mejor opción"""
    ahorros = {}
    for caso in Casos:
        node1, node2 = caso
        nueva_duracion_total, respuesta, ahorro = incompatibilidad(
            node1, node2, early_start, late_start, tiempos_proyecto, duracion_total_proyecto
        )
        ahorros[respuesta] = (ahorro, nueva_duracion_total)

    # Mostrar ahorros calculados para cada caso
    print("   Ahorros calculados:", ahorros)  # Debug: Muestra ahorros

    # Encontrar la opción con mayor ahorro
    mejor_opcion = max(ahorros, key=lambda x: ahorros[x][0])
    ahorro_maximo, nueva_duracion = ahorros[mejor_opcion]

    return mejor_opcion, nueva_duracion, ahorro_maximo

def modificar_grafo_y_recalcular(G, mejor_opcion, tiempos_proyecto):
    """Modifica el grafo según la mejor opción de precedencia y recalcula los tiempos"""
    # Modificar el grafo
    node1, node2 = mejor_opcion.split(' -> ')
    if mejor_opcion == f'{node1} -> {node2}':
        G.add_edge(node1, node2, weight=tiempos_proyecto[node2])  # Añadir la relación
    else:
        G.add_edge(node2, node1, weight=tiempos_proyecto[node1])  # Añadir la relación inversa

    # Recalcular los tiempos tras la modificación
    early_start, early_finish, late_start, late_finish, duracion_total_proyecto = recalcular_tiempos(G, tiempos_proyecto)

    return G, early_start, late_start, duracion_total_proyecto

# Configuración inicial del proyecto
precedencias = {
    'A': [], 'B': [], 'C': ['A'], 'D': ['A', 'B'], 'E': ['C'], 'F': ['D'], 'G': ['D'], 'H': ['E']
}

# Ingresar el valor de X para la actividad D
X = input("Ingrese el valor de X para la actividad D: ")
x = int(X)  # Convertir el valor ingresado a entero
tiempos_proyecto = {'A': 5, 'B': 7, 'C': 3, 'D': x, 'E': 2, 'F': 3, 'G': 5, 'H': 6}

# Crear el grafo del proyecto
G_proyecto = crear_grafo(precedencias, tiempos_proyecto)[0]
Camino_critico = crear_grafo(precedencias, tiempos_proyecto)[1]
if 'α' in Camino_critico:
    Camino_critico.remove('α')

# Recalcular los tiempos originales
early_start_ori, early_finish_ori, late_start_ori, late_finish_ori, duracion_total_proyecto_ori = recalcular_tiempos(G_proyecto, tiempos_proyecto)
print(f"1. Duración total del proyecto: {duracion_total_proyecto_ori} días, Camino crítico: {' -> '.join(Camino_critico)}")


# Evaluar incompatibilidades y realizar modificaciones
contador = 1  # Inicializar contador de opciones
Casos = [('C', 'D'), ('D', 'E'), ('F', 'G')]  # Definir los casos a evaluar

print(f'3. Evaluación incompatibilidad ...')
while Casos:
    # Evaluar el mejor caso de incompatibilidad
    mejor_opcion, nueva_duracion, ahorro_maximo = comp_incompatibilidad(Casos, G_proyecto, early_start_ori, late_start_ori, tiempos_proyecto, duracion_total_proyecto_ori)

    # Imprimir el resultado de la incompatibilidad con mayor ahorro
    letra_opcion = chr(96 + contador)  # a = 97 en ASCII
    print(f"   {letra_opcion}) \033[1mMejor opción de precedencia: {mejor_opcion}\033[0m")
    print(f"      \033[1mDuración nueva del proyecto: {nueva_duracion} días\033[0m")
    print(f"      Ahorro en la duración: {ahorro_maximo} días")

    # Modificar el grafo con la mejor opción y recalcular tiempos
    G_proyecto, early_start_ori, late_start_ori, duracion_total_proyecto_ori = modificar_grafo_y_recalcular(G_proyecto, mejor_opcion, tiempos_proyecto)

    # Eliminar el caso procesado de Casos
    node1, node2 = mejor_opcion.split(' -> ')
    Casos = [caso for caso in Casos if (node1, node2) != caso and (node2, node1) != caso]

    contador += 1



Ingrese el valor de X para la actividad D: 4
1. Duración total del proyecto: 16 días, Camino crítico: A -> C -> E -> H
3. Evaluación incompatibilidad ...
   Ahorros calculados: {'C -> D': (5, 17), 'D -> E': (0, 19), 'F -> G': (0, 19)}
   a) Mejor opción de precedencia: C -> D
      Duración nueva del proyecto: 17 días
      Ahorro en la duración: 5 días
   Ahorros calculados: {'E -> D': (1, 19), 'F -> G': (0, 20)}
   b) Mejor opción de precedencia: E -> D
      Duración nueva del proyecto: 19 días
      Ahorro en la duración: 1 días
   Ahorros calculados: {'F -> G': (0, 22)}
   c) Mejor opción de precedencia: F -> G
      Duración nueva del proyecto: 22 días
      Ahorro en la duración: 0 días


4. Determine el instante mínimo de finalización según las condiciones de precedencias,
sin tener presentes las incompatibilidades, y si sólo se dispone de dos equipos de
trabajo. Utilizar ambos esquemas (serie y paralelo), así como una ordenación de
actividades por número total de sucesoras (en caso de empate, utilizar el orden
alfabético).

In [ ]:
import networkx as nx

# Ejemplo de grafo con precedencias
precedencias_es = {'A': [], 'B': [], 'C': ['A'], 'D': ['A', 'B'], 'E': ['C'], 'F': ['D'], 'G': ['D'], 'H': ['E']}
# Ingresar el valor de X para la actividad D
x_2 = int(input("Ingrese el valor de X para la actividad D: "))  # Convertir el valor ingresado a entero
tiempos_proyecto_es = {'A': 5, 'B': 7, 'C': 3, 'D': x_2, 'E': 2, 'F': 3, 'G': 5, 'H': 6}

# Crear el grafo del proyecto
G_proyecto_esquema = nx.DiGraph()
# Agregar nodos y aristas al grafo según las precedencias
for tarea, predecesores in precedencias_es.items():
    G_proyecto_esquema.add_edges_from((predecesor, tarea) for predecesor in predecesores)

def contar_sucesores_hacia_atras(G, nodo, visitados=None):
    """Contar todos los sucesores de un nodo dado desde el final hacia atrás."""
    visitados = visitados or set()
    if nodo in visitados:
        return 0
    visitados.add(nodo)
    return sum(1 + contar_sucesores_hacia_atras(G, sucesor, visitados) for sucesor in G.successors(nodo))

def debug_sucesores_hacia_atras(G):
    """Imprimir el conteo total de sucesores desde el final de cada nodo en el grafo."""
    for nodo in G.nodes:
        if nodo not in ['α', 'ω']:  # Ignorar nodos ficticios
            print(f"Nodo {nodo} tiene un total de {contar_sucesores_hacia_atras(G, nodo)} sucesores hasta el final.")

# Ejecutar la función de debug
debug_sucesores_hacia_atras(G_proyecto_esquema)




Ingrese el valor de X para la actividad D: 2
Nodo A tiene un total de 6 sucesores hasta el final.
Nodo C tiene un total de 2 sucesores hasta el final.
Nodo D tiene un total de 2 sucesores hasta el final.
Nodo B tiene un total de 3 sucesores hasta el final.
Nodo E tiene un total de 1 sucesores hasta el final.
Nodo F tiene un total de 0 sucesores hasta el final.
Nodo G tiene un total de 0 sucesores hasta el final.
Nodo H tiene un total de 0 sucesores hasta el final.
